In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [ ]:
filename = 'Sonnets.txt'
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()
# print(raw_text)

In [ ]:
sonnets = raw_text.split('\n\n')
sonnet_lens = [len(sonnet) for sonnet in sonnets]

In [ ]:
# sonnets
# len(sonnet_lens)

In [ ]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c,i) for i, c in enumerate(chars))
int_to_char = dict((i,c) for i, c in enumerate(chars))
n_chars = len(raw_text)
n_vocab = len(chars)
print(n_chars)
print(n_vocab)
print(char_to_int)
print(int_to_char)

In [ ]:
# print(n_chars, n_vocab)

In [ ]:
# string = 'abcd'
# string[0:2]

In [ ]:
maxlen = 40
step = 3 #the step indicates how many characters to skip every time like from -> om which means to skip two charaters
sentences = []
next_chars = []
for i in range(0, n_chars-maxlen, step):
    sentences.append(raw_text[i:i+maxlen])
    next_chars.append(raw_text[i+maxlen])
print(len(sentences)) # no of sequences
print(len(next_chars)) # no of next charater of every senquence
print(n_chars-maxlen)
print(sentences)
print(next_chars)

In [ ]:
# print(sentences[:2])

In [ ]:
# for i,sentence in enumerate(sentences[:2]):
#   print("Value of i")
#   print(i)
#   print("value of sentence")
#   print(sentence)
#   for t, char in enumerate(sentence):
#     print("value of every t")
#     print(t)
#     print("value of every char")
#     print(char)


In [ ]:
#vectorization
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
# print(x.shape)
# print(y.shape)
for i,sentence in enumerate(sentences):
    # i is the index of every sentence in the sentences list,
    #snetence is every sentence in the sentences list
    for t, char in enumerate(sentence):
        x[i,t,char_to_int[char]] = 1
        #the x represents: i index of every sentence , t index of every character, char_to_int[char]
        # is the corresponding value for every character in the sentence
    y[i,char_to_int[next_chars[i]]] = 1
    # i is explained, char_to_int[next_chars[i]] is the value of next character given the current sentence

In [ ]:
x.shape

(32044, 40, 40)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.optimizers import RMSprop

model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               86528     
                                                                 
 dense (Dense)               (None, 40)                5160      
                                                                 
Total params: 91688 (358.16 KB)
Trainable params: 91688 (358.16 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# model = keras.models.Sequential([
#  keras.layers.GRU(128, return_sequences=True, input_shape=(maxlen, len(chars)),
#  dropout=0.2, recurrent_dropout=0.2),
#  keras.layers.GRU(128, return_sequences=False,
#  dropout=0.2, recurrent_dropout=0.2),
#  keras.layers.Dense(n_vocab,
#  activation="softmax")
# ])

# model.summary()
# model.compile(loss="categorical_crossentropy", optimizer=optimizer)
# #history = model.fit(dataset, epochs=20)

In [ ]:
# x.shape

In [ ]:
# print(x[0])

In [ ]:
history = model.fit(x, y,
          batch_size=128,
          epochs=50)

Epoch 1/50
251/251 [==============================] - 27s 100ms/step - loss: 2.4598
Epoch 2/50
251/251 [==============================] - 25s 98ms/step - loss: 1.9829
Epoch 3/50
251/251 [==============================] - 26s 103ms/step - loss: 1.8079
Epoch 4/50
251/251 [==============================] - 26s 102ms/step - loss: 1.6863
Epoch 5/50
251/251 [==============================] - 24s 97ms/step - loss: 1.5941
Epoch 6/50
251/251 [==============================] - 24s 96ms/step - loss: 1.5086
Epoch 7/50
251/251 [==============================] - 24s 96ms/step - loss: 1.4279
Epoch 8/50
251/251 [==============================] - 27s 106ms/step - loss: 1.3545
Epoch 9/50
251/251 [==============================] - 27s 106ms/step - loss: 1.2868
Epoch 10/50
251/251 [==============================] - 29s 115ms/step - loss: 1.2266
Epoch 11/50
251/251 [==============================] - 26s 102ms/step - loss: 1.1783
Epoch 12/50
251/251 [==============================] - 26s 102ms/step - loss: 

In [ ]:
model.save("./models/model_sp.h5")

In [ ]:
model.load_weights('model_sp.h5')

In [ ]:
def sample(preds, temperature=0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
def generate_sonnet(temp, ok_sonnet):
    ''' Given a temperature, generate a new sonnet '''
    #start_idx = np.random.randint(0, len(raw_text) - maxlen - 1)
    #new_sonnet = raw_text[start_idx:start_idx + maxlen]
    ok_sonnet += " why hath you been summoned at this bequet hour to mine chambers"
    new_sonnet = ok_sonnet[:maxlen]
    for i in range(600):
        # Vectorize generated text
        sampled = np.zeros((1, maxlen, len(chars)))
        for j, char in enumerate(new_sonnet):
            sampled[0, j, char_to_int[char]] = 1.

        # Predict next character
        preds = model.predict(sampled, verbose=0)[0]
        pred_idx = sample(preds, temperature=temp)
        next_char = chars[pred_idx]

        # Append predicted character to seed text
        new_sonnet += next_char
        new_sonnet = new_sonnet[1:]

        # Print to console
        print(next_char, end = '')

In [ ]:
ok_sonnet = " why hath you been summoned at this bequet hour to mine chambers"
new_sonnet = ok_sonnet[:maxlen]
sampled = np.zeros((1, maxlen, len(chars)))

In [ ]:
# for j, char in enumerate(new_sonnet):
#     sampled[0, j, char_to_int[char]] = 1.
# print(sampled)

In [ ]:
generate_sonnet(0.36, 'from')

mime.

lxxxviii

loo you love thee, of you are you troul love hild,
that mine is toon to hold thy praise.

lxx

the sicpering with as truth bring woodd,
when my bod the excest, and i evernce approve sheet word,
thy youthere his grace which thoughts all will.

xxv

then lof your some i love that so reep wound:
thy had my love should parts of thy dear with withit
det but you some glory thy dear tike,
though i made be be tume doth find wow,
or those varts thy poor doth with submery.

lxx

then some from her have thy sweet live younds
for if thou dart which the surm’dess an ellight
and you love th

In [ ]:
import pickle

with open('char_to_int.pkl','wb') as f:
    pickle.dump(char_to_int, f)

with open('int_to_char.pkl', 'wb') as f:
    pickle.dump(int_to_char, f)

with open('shakespeare.pkl', 'wb') as f:
    pickle.dump(model, f)
with open('shakespeare_chars.pkl','wb') as f:
    pickle.dump(chars, f)